Installs and Imports

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

**Sets**

* $\{1,2,\ldots,n\}$ = set of regions in the U.S. (indexed by $i$ or $j$)

**Parameters**

* $b_i =$  supply of counselors in region $i$ ($i = 1,2,\ldots,n$)
* $d_j =$  demand for counselors in region $j$ ($j = 1,2,\ldots,n$)
* $e =$ equitable care ratio. Given by $\frac{\sum_{i = 1}^n b_i}{\sum_{j = 1}^n d_j}$
* $c_{ij} = $ cost to incentivize a counselor from region $i$ to region $j$ ($i = 1,2,\ldots,n; \ j = 1,2,\ldots,n$)

**Decision Variables**

* $x_{ij} = $ number of counselors incentivized from region $i$ to region $j$ ($i = 1,2,\ldots,n; \ j = 1,2,\ldots,n$)

**Model**

$\begin{align}
\min \  & \sum_{i=1}^n \sum_{j = 1}^n c_{ij}x_{ij} & & \textrm{(counselor incentivization cost)}\\
\textrm{s.t.}\ & b_i + \sum_{j = 1}^n x_{ji} - \sum_{j = 1}^n x_{ij} \geq ed_i, \ \forall i = 1,2,\ldots,n && \textrm{(region $i$ meets the equitable care standard)}\\
&  x_{ij} \geq 0, \ \forall i = 1,2,\ldots,n && \textrm{(no negative flow)} \\
 &&
\end{align}$

$c_{ij} = $ (number of years we want to incentivize work for) $*$ (non-negative difference in income between region $i$ and region $j$) $+$ (constant incentive cost)

t * max(0, income[i] - income[j]) + a

In [ ]:
distress_df = pd.read_csv('distress_vs_resources.csv')
distress_df

,Unnamed: 0,mean_income,counselor_supply,cbsa,population,mental_health_demand
0,0,49235.00000,590,10180,346370,55479.655
1,2,52688.57143,2680,10420,1402898,228244.974
2,3,49563.33333,270,10500,296354,50270.225
3,4,64560.00000,290,10540,262108,41150.956
4,5,59085.00000,4010,10580,1757100,237785.436
...,...,...,...,...,...,...
363,368,57050.00000,1200,49420,503758,83371.949
364,369,51393.33333,1200,49620,900896,138287.536
365,370,50003.33333,2100,49660,1062840,178621.735
366,371,69288.33333,450,49700,353090,54823.405


In [ ]:
# set of regions, indexed by i
I = list(distress_df.index)

# constant incentive cost
a = 10000

# number of years that we are expecting counselors to stay in the region
t = 5

# supply of counselors in each region
b = {i: distress_df.at[i,'counselor_supply'] for i in I}

# demand for mental health in each region
d = {i: distress_df.at[i,'mental_health_demand'] for i in I}

# cost to send a counselor from region i to region j
# calculated by taking the nonnegative difference between the incomes of region
# i and region j plus a constant administrative cost
# formula: c[i, j] = t * max(0, income[i] - income[j]) + a
c = {(i,j): t * max(0, distress_df.at[i,'mean_income'] - distress_df.at[j,'mean_income']) + a for i in I for j in I}

# equitable standard of mental health care
e = sum(b.values()) / sum(d.values())

In [ ]:
# Create and Clear the model
model = pyo.ConcreteModel()
model.clear()

# Define index set for the regions
model.I = pyo.Set(initialize=I)

# Define supply in each region
model.b = pyo.Param(model.I, initialize=b)

# Define the demand in each region
model.d = pyo.Param(model.I, initialize=d)

# Cost of incentivizing from region i to region j
model.c = pyo.Param(model.I, model.I, initialize=c)

# Number of counselors incentivized from region i to region j
model.x = pyo.Var(model.I, model.I, domain=pyo.NonNegativeReals)

# Define the objective (minimizing cost)
def obj_rule(model):
  return(sum(model.c[i, j] * model.x[i, j] for i in model.I for j in model.I)) # total cost of shipping

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Define equitable care constraint
def equitable_care_rule(model, i):
  return b[i] + sum(model.x[j, i] - model.x[i, j] for j in model.I) >= e * d[i]

# include constraint for every region
model.equitable_care_requirement = pyo.Constraint(model.I, rule=equitable_care_rule)


### Solving the Model

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
opt.solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5367826871.0
  Upper bound: 5367826871.0
  Number of objectives: 1
  Number of constraints: 369
  Number of variables: 135425
  Number of nonzeros: 135056
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 6.07
  Wallclock time: 6.17
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None

In [ ]:
#Create new empty dataframe
df = pd.DataFrame({'from_cbsa':[], 'to_cbsa': [], 'counselors_transferred': []})

In [ ]:
#Access the variable values greater than 0 using pprint and save them to dataframe
for i in I:
  for j in I:
    if model.x[i,j].value > 0:
      #model.x[i,j].pprint()
      df.loc[len(df.index)] = [distress_df.at[i,'cbsa'], distress_df.at[j,'cbsa'], model.x[i,j].value]

In [ ]:
print('objective value = ', pyo.value(model.obj))

objective value =  5367826877.467677


In [ ]:
df.to_csv('CBSA.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# look at top incoming states
df.groupby('to_cbsa')['counselors_transferred'].agg([np.sum]).sort_values(by=['sum'], ascending=False)

In [ ]:
# look at top outcoming states
df.groupby('from_cbsa')['counselors_transferred'].agg([np.sum]).sort_values(by=['sum'], ascending=False)